# 7기 마에스트로 백엔드 과제 - 상품 카테고리 자동 분류 서버 

# 과제 개요

 * 출제자 : 남상협 멘토 (justin@buzzni.com) / 버즈니 (http://buzzni.com) 대표 
 * 배경 : 카테고리 분류 엔진은 실제로 많은 서비스에서 사용되는 중요한 기계학습의 한 분야이다. 본 과제는 버즈니 개발 인턴이자 마에스트로 6기 멘티가 아래와 나와 있는 기본 분류 모델을 기반으로 deep learning 기반의 feature 를 더해서 고도화된 분류 엔진을 만들어서 2016 한국 정보과학회 논문으로도 제출 했던 과제다. 기계학습에 대한 학습과, 실용성 두가지 측면에서 모두 도움이 될 것으로 보인다.


## 과제 목표

 * 입력 : 상품명, 상품 이미지
 * 출력 : 카테고리
 * 목표 : 가장 높은 정확도로 분류를 하는 분류 엔진을 개발

## 제약 조건
 * 분류 엔진은 rest api 형태로 만들어야함 (샘플로 준 python server 사용해도 됨)
 * rest api 는  http://서버주소:포트/classify?name=상품명&img=상품이미지주소 형식으로 호출이 가능해야함
 * 위 rest api 를 이용해서 별도로 가지고 있는 데이터로 자동 성능 평가로 채점을 하게됨 (채점하는 데이터는 제공되지 않음)
 * 데이터 리턴 형식은 {u'cate': u'\ud328\uc158\uc758\ub958;\uc544\ub3d9\uc758\ub958;\ud55c\ubcf5'} 이런식으로 cate 라는 키 값에 그에 해당되는 대,중,소 카테고리를 ; 로 연결한 형태로 반환하면 됨 

## 평가 항목
 1. 성능 평가 (100%)

## 제출 항목
 1. 채점 서버 호출이 필요함: 채점 서버 호출시 name 부분에 자신의 이름을 넣으면 됨.
  - 자신이 개발한 서버 호출 형태 : http://서버주소:포트/classify?name=상품명&img=상품이미지주소
  - 평가 서버 : http://somaeval.hoot.co.kr:8880/eval?url= 뒤에 자신의 서버 주소를 넣으면 됨 너무 자주 호출하면 서버가 죽을 수 있으니, mode=all 로는 꼭 필요할때만 호출하기 바람 , 평가 서버가 죽었을 시에는 justin@buzzni.com 으로 문의 
  - 예) 샘플 테스트 :  http://somaeval.hoot.co.kr:8880/eval?url=http://somaeval.hoot.co.kr:18887
  - 예) 전체 테스트 : http://somaeval.hoot.co.kr:8880/eval?url=http://somaeval.hoot.co.kr:18887&mode=all&name=베이스라인

## 성능 향상 포인트
 * 오픈된 형태소 분석기(예 - 은전한닢 http://eunjeon.blogspot.kr/ )를 써서, 단어 띄어쓰기를 의미 단위로 띄어서 학습하기 
 * 상품명에서 분류하는데 도움이 되지 않는 stop word 제거하기 
 * bigram, unigram, trigram 등 단어 feature 를 더 다양하게 추가하기 
 * 이미지 데이터를 Deep Learning (CNN) 기반 방법으로 feature 를 추출해서 추가하기 
  * 제일 기본적으로 https://github.com/irony/caffe-docker-classifier 이런 이미 만들어진 모델을 이용해서 feature 를 추출해서 추가하기도 가능함 
  * DIGITS + caffe 를 이용해서 본 학습 모델에 맞는 이미지 자동 분류기를 별도로 학습해서 사용하는것도 가능함

## 개발 서버 URL 형태 
 * name, img 를 파리미터로 호출한다.
 * 호출 형태 : http://somaeval.hoot.co.kr:18887/classify?name=조끼&img=http://shopping.phinf.naver.net/main_8134935/8134935099.1.jpg 
 

## 성능평가 테스트 서버 
 * 아래 주소에서 url 에 자신의 분류기 모델 주소를 넣어주면 제공되지 않았던 데이터들을 이용해서 평가를 해준다.
 * 샘플 테스트(카테고리별 2개만 가지고 테스트) : http://somaeval.hoot.co.kr:8880/eval?url=http://somaeval.hoot.co.kr:18887
 * 전체 성능 평가 테스트 (mode=all) : http://somaeval.hoot.co.kr:8880/eval?url=http://somaeval.hoot.co.kr:18887&mode=all
 

## 현재 보고 있는 IPython Notebook 사용법
 * https://www.youtube.com/results?search_query=ipython+notebook+tutorial
 * shift + enter 를 누르면 실행이 된다.

# 아래는 baseline 모델을 만드는 방법 

In [1]:
import pandas as pd # python에서 데이터를 조작할 때 쓰는 툴

## 데이터를 읽는다.
 * 아래 id 에 해당되는 이미지 데이터 다운받기 https://www.dropbox.com/s/q0qmx3qlc6gfumj/soma_train.tar.gz
 

In [2]:
train_df = pd.read_pickle("soma_goods_train.df")

 * cate1 는 대분류, cate2 는 중분류, cate3 는 소분류 
 * 총 10000개의 학습 데이터

In [3]:
train_df.shape

(10000, 4)

In [4]:
train_df[0:3]

,cate1,cate2,cate3,name
90985,디지털/가전,PC부품,CPU,인텔 인텔 코어i7-4세대 4770K (하스웰) (정품)
90986,디지털/가전,PC부품,CPU,애플 맥북에어 13형 MacBook Air 13.3/1.6/4/256FLASH MJ...
90987,디지털/가전,PC부품,CPU,애플 맥북에어 11형 MacBook Air 11.6/1.6/4/128FLASH MJ...


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

 * CountVectorizer 는 일반 text 를 이에 해당되는 숫자 id 와, 빈도수 형태의 데이터로 변환 해주는 역할을 해준다.
 * 이 역할을 하기 위해서 모든 단어들에 대해서 id 를 먼저 할당한다.
 * 그리고 나서, 학습 데이터에서 해당 단어들과, 그것의 빈도수로 데이터를 변환 해준다. (보통 이런 과정을 통해서 우리가 이해하는 형태를 컴퓨터가 이해할 수 있는 형태로 변환을 해준다고 보면 된다)
 * 예를 들어서 '베네통키즈 키즈 러블리 키즈' 라는 상품명이 있고, 각 단어의 id 가 , 베네통키즈 - 1, 키즈 - 2, 러블리 -3 이라고 한다면 이 상품명은 (1,1), (2,2), (3,1) 형태로 변환을 해준다. (첫번째 단어 id, 두번째 빈도수)
 

In [6]:
vectorizer = CountVectorizer()

 * 대분류, 중분류, 소분류 카테고리 명을 합쳐서 카테고리명을 만든다.  우리는 이 카테고리명을 예측하는 분류기를 만들게 된다.
 * d_list 에는 학습하는 데이터(상품명) 을 넣고, cate_list 에는 분류를 넣는다.

In [7]:
## 특정 번호의 상품 조회
number = 872411
part = train_df
for each in part.iterrows():
    if each[0] == number:
        print each[1]
#print each[0]
#    print each[1]['name']

cate1                                                 패션의류
cate2                                                 여성의류
cate3                                                  파티복
name     [해외] Free Shipping women&#39;s low-waist vinta...
Name: 872411, dtype: object


In [8]:
d_list = []
cate_list = []
for each in train_df.iterrows():
    cate = ";".join([each[1]['cate1'],each[1]['cate2'],each[1]['cate3']])
    d_list.append(each[1]['name'])
    cate_list.append(cate)

In [9]:
##
## print set(cate_list)
print d_list[0:1]
print cate_list[0:1]
#print set(cate_list)

[u'\uc778\ud154 \uc778\ud154 \ucf54\uc5b4i7-4\uc138\ub300 4770K (\ud558\uc2a4\uc6f0) (\uc815\ud488)']
[u'\ub514\uc9c0\ud138/\uac00\uc804;PC\ubd80\ud488;CPU']


In [12]:
#print len(set(cate_list))
# print set(cate_list)

#practice

 * 각 카테고리명에 대해서 serial 한 숫자 id 를 부여한다.
 * cate_dict[카테고리명] = serial_id 형태이다. 

In [13]:
cate_dict = dict(zip(list(set(cate_list)),range(len(set(cate_list))))) # 카테고리 별로 id를 만들어주는 코드.

In [14]:
print cate_dict[u'디지털/가전;네트워크장비;KVM스위치']
print cate_dict[u'패션의류;남성의류;정장']

0
80


 * y_list 에는 단어 형태의 카테고리명에 대응되는 serial_id 값들을 넣어준다.

In [15]:
# 저장해놓은 거 미리 읽어오기
from sklearn.externals import joblib
list_caffe = []
list_caffe = joblib.load('list_caffe.dat')
length = len(list_caffe)
print length

10000


In [58]:
# test
test_num = 7554
res = list_caffe[test_num-1]['result']
a = d_list[test_num]
if res[0] :
    for each in res[2] :
        a += " " + each[0].replace(" ", "")

print a
# print train_df[test_num-1:test_num]

[올리브데올리브][올리브데올리브]OW5SB119오버핏 기본카라 블라우스 clothing skirt consumergoods garment commodity


In [75]:
# google net 써서 json 받아오기
'''
import requests

url_caffe = 'http://192.168.99.100:5000/classify_url?imageurl=https://raw.githubusercontent.com/YeongjinOh/training_images/master/%s.jpg'
list_caffe = joblib.load('list_caffe.dat')
length = len(list_caffe)
i = length
print "start from %d"%i


more = 10000
for each in train_df[length:length+more].iterrows(): # iterrows 데이터를 도는 애
    response = requests.get(url_caffe%each[0])
    try :
        result_caffe = response.json()
    except :
        result_caffe = {u'result': [False]}
        print "except %d"%i
    list_caffe.append(result_caffe)
    i = i + 1

    if i % 10 == 0:
        joblib.dump(list_caffe,'list_caffe.dat',compress=3)
        print "save %d" % i
# save
joblib.dump(list_caffe,'list_caffe.dat',compress=3)
'''

'\nimport requests\n\nurl_caffe = \'http://192.168.99.100:5000/classify_url?imageurl=https://raw.githubusercontent.com/YeongjinOh/training_images/master/%s.jpg\'\nlist_caffe = joblib.load(\'list_caffe.dat\')\nlength = len(list_caffe)\ni = length\nprint "start from %d"%i\n\n\nmore = 10000\nfor each in train_df[length:length+more].iterrows(): # iterrows \xeb\x8d\xb0\xec\x9d\xb4\xed\x84\xb0\xeb\xa5\xbc \xeb\x8f\x84\xeb\x8a\x94 \xec\x95\xa0\n    response = requests.get(url_caffe%each[0])\n    try :\n        result_caffe = response.json()\n    except :\n        result_caffe = {u\'result\': [False]}\n        print "except %d"%i\n    list_caffe.append(result_caffe)\n    i = i + 1\n\n    if i % 10 == 0:\n        joblib.dump(list_caffe,\'list_caffe.dat\',compress=3)\n        print "save %d" % i\n# save\njoblib.dump(list_caffe,\'list_caffe.dat\',compress=3)\n'

In [64]:
# list_caffe_samples = []
# joblib.dump(list_caffe_samples,'list_caffe_samples.dat')

['list_caffe_samples.dat']

In [71]:
# Sample data 이용 : google net 써서 json 받아오기

import requests

# initialize
url_caffe = 'http://192.168.99.100:5000/classify_url?imageurl=%s'
list_caffe_samples = joblib.load('list_caffe_samples.dat')
length = len(list_caffe_samples)
print "%d samples are stored"% length

"""
# read image
img_list = joblib.load('img_list_sample.dat')
size = len(img_list)
print size

for i in range(0,size):
    response = requests.get(url_caffe%img_list[i])
    try :
        result_caffe_sample = response.json()
    except :
        result_caffe_sample = {u'result': [False]}
        print "except %d"%i
    list_caffe_samples.append(result_caffe_sample)
    joblib.dump(list_caffe_samples,'list_caffe_samples.dat')
    if i % 10 == 0:
        print "save %d" % i

print len(list_caffe_samples)
"""

206 samples are stored


'\n# read image\nimg_list = joblib.load(\'img_list_sample.dat\')\nsize = len(img_list)\nprint size\n\nfor i in range(0,size):\n    response = requests.get(url_caffe%img_list[i])\n    try :\n        result_caffe_sample = response.json()\n    except :\n        result_caffe_sample = {u\'result\': [False]}\n        print "except %d"%i\n    list_caffe_samples.append(result_caffe_sample)\n    joblib.dump(list_caffe_samples,\'list_caffe_samples.dat\')\n    if i % 10 == 0:\n        print "save %d" % i\n\nprint len(list_caffe_samples)\n'

In [69]:
print list_caffe_samples[0]

{u'result': [True, [[u'web site', u'0.56633'], [u'Band Aid', u'0.12045'], [u'packet', u'0.04885'], [u'cash machine', u'0.04730'], [u'envelope', u'0.04477']], [[u'web site', u'2.22455'], [u'computer', u'1.63205'], [u'machine', u'1.38676'], [u'device', u'0.50383'], [u'container', u'0.17822']], u'5.054']}


In [40]:
y_list = []
for each in train_df.iterrows(): # iterrows 데이터를 도는 애
    cate = ";".join([each[1]['cate1'],each[1]['cate2'],each[1]['cate3']])
    y_list.append(cate_dict[cate])

 * fit_transform 을 하게 되면, d_list 에 들어 있는 모든 단어들에 대해서, 단어-id 사전을 만드는 일을 먼저하고 (fit)
 * 실제로 d_list 에 들어 있는 각 데이터들에 대해서 (단어id,빈도수) 형태의 데이터로 변환을 해준다. (transform)

In [41]:
print d_list[0:1]
print y_list[0:1]
print vectorizer.fit_transform(d_list[0:1])
x_list = vectorizer.fit_transform(d_list)

[u'\uc778\ud154 \uc778\ud154 \ucf54\uc5b4i7-4\uc138\ub300 4770K (\ud558\uc2a4\uc6f0) (\uc815\ud488)']
[44]
  (0, 2)	2
  (0, 4)	1
  (0, 1)	1
  (0, 0)	1
  (0, 5)	1
  (0, 3)	1


In [49]:
print x_list.shape

(10000, 25767)


In [29]:
print y_list[0:3] # 카테고리명에 대응되는 serial_id 값들을 넣어준다.

[44, 44, 44]


 * 여기서는 분류에서 가장 많이 사용하는 SVM(Support Vector Machine) 을 사용한 분류 학습을 한다. 

In [30]:
from sklearn.svm import LinearSVC

In [31]:
from sklearn.grid_search import GridSearchCV

In [32]:
import numpy as np

In [33]:
svc_param = {'C':np.logspace(-2,0,20)} # 엔진이 학습을 할 때 파라미터에 따라서 최적화를 해준다. 속도를 빠르게 하고싶으면 (-2,0,5)로 바꿈.

 * grid search 를 통해서 최적의 c 파라미터를 찾는다.
 * 5 cross validation 을 한다. (cv 값 굳이 바꿀 필요 없음.)
 print svc_param

In [34]:
gs_svc = GridSearchCV(LinearSVC(loss='l2'),svc_param,cv=5,n_jobs=4) # n_jobs 몇개 코어로 돌릴거냐
# print x_list[0]
# print y_list[0]

In [35]:
print gs_svc

GridSearchCV(cv=5, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='l2', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'C': array([ 0.01   ,  0.01274,  0.01624,  0.02069,  0.02637,  0.0336 ,
        0.04281,  0.05456,  0.06952,  0.08859,  0.11288,  0.14384,
        0.1833 ,  0.23357,  0.29764,  0.37927,  0.48329,  0.61585,
        0.78476,  1.     ])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)


In [36]:
gs_svc.fit(x_list, y_list)

/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/svm/classes.py:197: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' will be removed in 1.0
  DeprecationWarning)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/svm/classes.py:197: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' will be removed in 1.0
  DeprecationWarning)
/opt/conda/envs/python2/lib/python2.7/site-packages/sklearn/svm/classes.py:197: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility f

GridSearchCV(cv=5, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='l2', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'C': array([ 0.01   ,  0.01274,  0.01624,  0.02069,  0.02637,  0.0336 ,
        0.04281,  0.05456,  0.06952,  0.08859,  0.11288,  0.14384,
        0.1833 ,  0.23357,  0.29764,  0.37927,  0.48329,  0.61585,
        0.78476,  1.     ])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

* 현재 기본 baseline 성능은 64% 정도로 나온다. 이 성능을 높이는 것이 본 과제의 목표이다. 
 * 위 grid search 로는 c 값을 찾고, 이렇게 찾은 c 값으로 다시 train 을 해서 최종 모델을 만든다.

In [42]:
print gs_svc.best_params_, gs_svc.best_score_
# overfitting underfitting 때문에.

{'C': 0.088586679041008226} 0.6418


In [43]:
clf = LinearSVC(C=gs_svc.best_params_['C'])
# C 값이 overfitting을 결정한다.

In [44]:
clf.fit(x_list,y_list)  # fit 하면 학습이 됨.

LinearSVC(C=0.088586679041008226, class_weight=None, dual=True,
     fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
     max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
     tol=0.0001, verbose=0)

In [45]:
from sklearn.externals import joblib

 * 만들어진 모델을 나중에도 쓰기 위해서 파일에 저장한다.
 * 아래 형태로 저장하면, 추후에 손쉽게 load 할 수 있다.
 * 이때 SVM 모델,  cate_name - cate_id 사전, 단어 - 단어_id,빈도수 변ㅎ

In [46]:
joblib.dump(clf,'classify.model',compress=3)   # 학습 데이터가 저장되어 있음
joblib.dump(cate_dict,'cate_dict.dat',compress=3)
joblib.dump(vectorizer,'vectorizer.dat',compress=3)

['vectorizer.dat']

 * 여기까지 모델을 만든다음에 classify_server 노트북을 열고, 쭉 실행을 시키면 서버가 뜬다.
 * 서버가 뜨면 아래처럼 실행을 시킬 수 가 있다.

In [47]:
import requests

In [48]:
name='[신한카드5%할인][예화-좋은아이들] 아동한복 여아 1076 빛이나노랑'
img=''

In [49]:
u='http://192.168.99.100:8887/classify?name=%s&img=%s'

In [50]:
r = requests.get(u%(name,img)).json()
# classify_server 이 노트북을 먼저 실행하고 나서 해야 동작한다.

In [51]:
print r

{u'cate': u'\ud328\uc158\uc758\ub958;\uc544\ub3d9\uc758\ub958;\ud55c\ubcf5'}
